<a href="https://colab.research.google.com/github/angeles1998-cell/Final_Project_BD/blob/main/finalproject_group5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import count, col, sum, round
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Creating a new session in Spark. This is an entry point to your application.

sc = SparkSession.Builder().appName("Final project").getOrCreate()

In [3]:
#Creating dataFrame and Join
df2 = sc.read.csv("/content/sample_data/job_skills.csv", header=True, inferSchema=True)
df=sc.read.csv("/content/sample_data/linkedin_job_postings.csv", header=True, inferSchema=True)


In [4]:
df.count()

1348488

In [6]:
len(df.columns) #  number of columns in the DataFrame

14

In [7]:
df.count() # number of rows in the DataFrame

1348488

In [5]:
#Verify the schema of Spark DataFrame
df.printSchema()

root
 |-- job_link: string (nullable = true)
 |-- last_processed_time: string (nullable = true)
 |-- got_summary: string (nullable = true)
 |-- got_ner: string (nullable = true)
 |-- is_being_worked: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- company: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- first_seen: string (nullable = true)
 |-- search_city: string (nullable = true)
 |-- search_country: string (nullable = true)
 |-- search_position: string (nullable = true)
 |-- job_level: string (nullable = true)
 |-- job_type: string (nullable = true)



In [24]:
#Summary statistics
df.describe().show()

+-------+--------------------+--------------------+--------------------+----------+---------------+--------------------+-----------------+--------------------+-------------+-----------+--------------+----------------+--------------------+---------+
|summary|            job_link| last_processed_time|         got_summary|   got_ner|is_being_worked|           job_title|          company|        job_location|   first_seen|search_city|search_country| search_position|           job_level| job_type|
+-------+--------------------+--------------------+--------------------+----------+---------------+--------------------+-----------------+--------------------+-------------+-----------+--------------+----------------+--------------------+---------+
|  count|             1348488|             1348488|             1348488|   1348488|        1348488|             1348488|          1348443|             1348435|      1348454|    1348420|       1348420|         1348420|             1348420|  1348419|
|   

In [10]:
df = df.join(df2, 'job_link', 'left')
df.show()

+--------------------+--------------------+-----------+-------+---------------+--------------------+--------------------+--------------------+----------+---------------+--------------+--------------------+----------+--------+--------------------+--------------------+
|            job_link| last_processed_time|got_summary|got_ner|is_being_worked|           job_title|             company|        job_location|first_seen|    search_city|search_country|     search_position| job_level|job_type|          job_skills|          job_skills|
+--------------------+--------------------+-----------+-------+---------------+--------------------+--------------------+--------------------+----------+---------------+--------------+--------------------+----------+--------+--------------------+--------------------+
|https://au.linked...|2024-01-21 02:01:...|          t|      t|              f|Supervisor - Hort...|Green Options Lan...|Sydney, New South...|2024-01-15|New South Wales|     Australia|  Booking Su

In [38]:
b= df.select("job_level").agg(count("job_level")).alias("Frecuency").groupBy("job_level")

AttributeError: 'GroupedData' object has no attribute 'show'

In [25]:
#Check unique values in each column (try with each column)
df.select("job_level").distinct().show()

+--------------------+
|           job_level|
+--------------------+
|              Onsite|
|           Associate|
|Chief Computer Pr...|
|          Mid senior|
|Job Development S...|
|           Inspector|
|              Canada|
|           Assembler|
|   Medical Physicist|
|              Lawyer|
|      Budget Analyst|
|        Order Runner|
|           Architect|
|     Nurse Assistant|
|Nurse Infection C...|
|      Microbiologist|
|      Model Artists'|
|     Color Developer|
|Administrative As...|
| Magazine Supervisor|
+--------------------+
only showing top 20 rows



In [ ]:
#Check if we have any duplicate value
df.dropDuplicates().show()

In [29]:
b= df.groupBy('job_level').agg(count('job_level').alias('Frecuency')). show(30)

+--------------------+---------+
|           job_level|Frecuency|
+--------------------+---------+
|              Onsite|        1|
|                NULL|        0|
|           Associate|   143997|
|Chief Computer Pr...|        1|
|          Mid senior|  1204393|
|Job Development S...|        1|
|           Inspector|        1|
|              Canada|        1|
|           Assembler|        1|
|   Medical Physicist|        1|
|              Lawyer|        1|
|      Budget Analyst|        1|
|        Order Runner|        1|
|           Architect|        1|
|     Nurse Assistant|        1|
|Nurse Infection C...|        1|
|      Microbiologist|        1|
|      Model Artists'|        1|
|     Color Developer|        1|
|Administrative As...|       10|
| Magazine Supervisor|        1|
|       United States|        1|
|  Investment Analyst|        1|
|Documentation Sup...|        1|
+--------------------+---------+



In [32]:
spark = SparkSession.builder.appName("Job_Spark").getOrCreate()

In [33]:
df2 = spark.read.csv("/content/sample_data/job_skills.csv", header=True, inferSchema=True)
df=spark.read.csv("/content/sample_data/linkedin_job_postings.csv", header=True, inferSchema=True)
df = df.join(df2, 'job_link', 'left')

In [39]:
df.printSchema()

root
 |-- job_link: string (nullable = true)
 |-- last_processed_time: string (nullable = true)
 |-- got_summary: string (nullable = true)
 |-- got_ner: string (nullable = true)
 |-- is_being_worked: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- company: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- first_seen: string (nullable = true)
 |-- search_city: string (nullable = true)
 |-- search_country: string (nullable = true)
 |-- search_position: string (nullable = true)
 |-- job_level: string (nullable = true)
 |-- job_type: string (nullable = true)
 |-- job_skills: string (nullable = true)



In [40]:
df.createOrReplaceTempView("df")

In [51]:
spark.sql("select job_title, company from df order by company desc ").show()

+--------------------+--------------------------+
|           job_title|                   company|
+--------------------+--------------------------+
|        Camp Manager|           🚀 Creator Camp|
|Customer Service ...|      👠 Cinderellas Cl...|
|Customer Service ...|      👔 Shaffer Dry Cl...|
|Maintenance Super...|      🌟Daniel-Scott Re...|
|          IT Manager|      🌟Daniel-Scott Re...|
|Assistant Practit...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Speciality Doctor...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Consultant Physic...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|   Contracts Manager|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Consultant Urolog...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Building Maintena...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|         Ward Sister|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Band 6 Team Leade...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Occupational Heal...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Specialty Dr Opht...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Registered Staff ...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|  Department Manager|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Registered Staff ...|      𝗦𝘁𝗼𝗰𝗸𝗽𝗼𝗿?...|
|Sales & Market De...|한화첨단소재 / Hanwha A...|
|Compensation B

In [94]:
# jobs with high demand
a_df= spark.sql("""
          select job_title,
          count(*) as Frecuency
          from df
          group by job_title
          order by Frecuency desc
          """)
a_df.show()

+--------------------+---------+
|           job_title|Frecuency|
+--------------------+---------+
|LEAD SALES ASSOCI...|     7325|
|       Shift Manager|     5818|
|First Year Tax Pr...|     5356|
|   Assistant Manager|     5346|
|Customer Service ...|     5203|
|LEAD SALES ASSOCI...|     4924|
|       Store Manager|     4791|
|CUSTOMER SERVICE ...|     4218|
|    Registered Nurse|     4190|
|Hourly Supervisor...|     2955|
|Travel Allied Hea...|     2933|
|                Host|     2871|
|   Account Executive|     2631|
|   Senior Accountant|     2548|
|  Restaurant Manager|     2335|
|Hourly Supervisor...|     2230|
|        Veterinarian|     2200|
|OPERATIONS ASSIST...|     2066|
|Assistant General...|     2054|
| Executive Assistant|     2054|
+--------------------+---------+
only showing top 20 rows



In [107]:
df.plot.scatter(x='job_title', y='Frecuency')


AttributeError: 'DataFrame' object has no attribute 'plot'

In [57]:
spark.sql("""
          select company,
          count(*) as Frecuency
          from df
          group by company
          order by Frecuency desc
          """).show()

+--------------------+---------+
|             company|Frecuency|
+--------------------+---------+
|     Health eCareers|    41597|
|   Jobs for Humanity|    27680|
|   TravelNurseSource|    16142|
|      Dollar General|    14815|
|        PracticeLink|     9737|
|      Energy Jobline|     9364|
|Gotham Enterprise...|     8935|
|               Jobot|     8713|
|       ClearanceJobs|     8599|
|          McDonald's|     8125|
|      VolunteerMatch|     6833|
|             Walmart|     6629|
|             DocCafe|     6106|
|           H&R Block|     5697|
|            Circle K|     5572|
|        ClickJobs.io|     5570|
|         CyberCoders|     5401|
|        Michael Page|     4737|
|U.S. Department o...|     4564|
| BCG Attorney Search|     4558|
+--------------------+---------+
only showing top 20 rows



In [62]:
spark.sql("""
          select job_level,
          count(*) as Frecuency
          from df
          group by job_level
          order by Frecuency desc
          """).show()

+--------------------+---------+
|           job_level|Frecuency|
+--------------------+---------+
|          Mid senior|  1204393|
|           Associate|   143997|
|                NULL|       68|
|Administrative As...|       10|
|           Assembler|        1|
|              Onsite|        1|
|   Medical Physicist|        1|
|       United States|        1|
|Chief Computer Pr...|        1|
|           Architect|        1|
|     Nurse Assistant|        1|
|Nurse Infection C...|        1|
|           Inspector|        1|
|Documentation Sup...|        1|
|              Lawyer|        1|
|      Budget Analyst|        1|
|        Order Runner|        1|
|Job Development S...|        1|
|  Investment Analyst|        1|
|      Microbiologist|        1|
+--------------------+---------+
only showing top 20 rows



In [63]:
spark.sql("""
          select job_location,
          count(*) as Frecuency
          from df
          group by job_location
          order by Frecuency desc
          """).show()

+--------------------+---------+
|        job_location|Frecuency|
+--------------------+---------+
|        New York, NY|    13435|
|London, England, ...|    11756|
|         Houston, TX|    10999|
|         Chicago, IL|    10790|
|     Los Angeles, CA|    10262|
|         Atlanta, GA|     9208|
|          Boston, MA|     8403|
|          Dallas, TX|     8005|
|          Austin, TX|     7678|
|      Washington, DC|     7322|
|         Phoenix, AZ|     7058|
|       San Diego, CA|     6883|
|       Charlotte, NC|     6774|
|    Philadelphia, PA|     6685|
|         Seattle, WA|     6584|
|     San Antonio, TX|     6419|
|   San Francisco, CA|     6014|
|           Tampa, FL|     5896|
|        Columbus, OH|     5798|
|       Las Vegas, NV|     5691|
+--------------------+---------+
only showing top 20 rows



In [68]:
spark.sql("""
          select search_country,
          count(*) as Frecuency
          from df
          group by search_country
          order by Frecuency desc
          """).show()
#The most important countries are  United States|United Kingdom| Canada|Australia

+------------------+---------+
|    search_country|Frecuency|
+------------------+---------+
|     United States|  1149310|
|    United Kingdom|   113391|
|            Canada|    55970|
|         Australia|    29719|
|              NULL|       68|
|             Akron|        7|
|          Columbus|        2|
|      Fayetteville|        2|
| Fort Walton Beach|        1|
|           Beverly|        1|
|         Oceanside|        1|
|          Arkansas|        1|
|        Alexandria|        1|
|         Nashville|        1|
|        2024-01-16|        1|
|Clinical Therapist|        1|
|            Layton|        1|
|         Tarrytown|        1|
|         Hollywood|        1|
|     North Chicago|        1|
+------------------+---------+
only showing top 20 rows



In [67]:
spark.sql("""
          select search_country,
          company,
          job_level,
          search_position
          from df
          group by search_country,company, job_level, search_position
          order by search_country desc
          """).show()

+--------------+--------------------+----------+--------------------+
|search_country|             company| job_level|     search_position|
+--------------+--------------------+----------+--------------------+
| United States|                 LHH| Associate|       Payroll Clerk|
| United States|AGI - Ag Growth I...|Mid senior|    Credit Counselor|
| United States|      Dollar General|Mid senior|Consultant Education|
| United States|      Baker Tilly US| Associate|               Baker|
| United States|        Aston Carter|Mid senior|Supervisor Line D...|
| United States|U.S. Department o...|Mid senior|Counselor Marriag...|
| United States|Challenge Manufac...|Mid senior|         Leader Tier|
| United States|           GLC Group|Mid senior|Laboratory Superv...|
| United States| Steward Health Care|Mid senior|Laboratory Superv...|
| United States|               Jobot|Mid senior|Superintendent Se...|
| United States|        EY-Parthenon|Mid senior|Director Translation|
| United States|Bob'

In [76]:
#The companies and the job level en United States, usually they are looking for Med Senior level
spark.sql("""
          select company as Company_in_US,
          job_level,
          count(search_position) as Frecuency
          from df
          where search_country = 'United States'
          group by search_country,company, job_level
          order by Frecuency desc
          """).show()

+--------------------+----------+---------+
|       Company_in_US| job_level|Frecuency|
+--------------------+----------+---------+
|     Health eCareers|Mid senior|    41451|
|   Jobs for Humanity|Mid senior|    27135|
|   TravelNurseSource|Mid senior|    16018|
|      Dollar General|Mid senior|    14756|
|        PracticeLink|Mid senior|     9714|
|Gotham Enterprise...|Mid senior|     8877|
|               Jobot|Mid senior|     8687|
|       ClearanceJobs|Mid senior|     8597|
|          McDonald's|Mid senior|     8074|
|      VolunteerMatch| Associate|     6823|
|             Walmart|Mid senior|     6167|
|             DocCafe|Mid senior|     6098|
|           H&R Block|Mid senior|     5690|
|         CyberCoders|Mid senior|     5382|
|            Circle K|Mid senior|     5377|
|      Energy Jobline|Mid senior|     4921|
|U.S. Department o...|Mid senior|     4553|
| BCG Attorney Search|Mid senior|     4535|
|       Family Dollar|Mid senior|     4333|
|              Target|Mid senior

In [74]:
#The companies and the job level en United kingdom, usually they are looking for Med Senior and Associate
spark.sql("""
          select company as Company_in_UK,
          job_level,
          count(search_position) as Frecuency
          from df
          where search_country = 'United Kingdom'
          group by search_country,company, job_level
          order by Frecuency desc
          """).show()

+--------------------+----------+---------+
|       Company_in_UK| job_level|Frecuency|
+--------------------+----------+---------+
|      Energy Jobline|Mid senior|     3924|
| Sanctuary Personnel| Associate|     1408|
|        Michael Page|Mid senior|     1403|
|Public Practice R...|Mid senior|     1248|
|Somerset NHS Foun...|Mid senior|     1170|
|IRIS | Networx | ...|Mid senior|      896|
|Your World Recrui...|Mid senior|      829|
|Recruitment Panda...|Mid senior|      783|
|           TipTopJob|Mid senior|      762|
|     Prospect Health|Mid senior|      743|
|          ecocareers|Mid senior|      702|
|Jupiter Recruitme...|Mid senior|      700|
|                Hays|Mid senior|      693|
|      Page Personnel|Mid senior|      659|
|         BMJ Careers|Mid senior|      556|
|   Turner & Townsend|Mid senior|      552|
|        ClickJobs.io|Mid senior|      512|
|       Academics Ltd|Mid senior|      466|
|       Guardian Jobs|Mid senior|      458|
|Bennett & Game Re...|Mid senior

In [78]:
#The companies and the job level in Canada, usually they are looking for Med Senior and Associate
#the most relevant comanies are CareerBeacon, ClickJobs and Canadian Armed Force
spark.sql("""
          select company as Company_in_Canada,
          job_level,
          count(search_position) as Frecuency
          from df
          where search_country = 'Canada'
          group by search_country,company, job_level
          order by Frecuency desc
          """).show()

+--------------------+----------+---------+
|   Company_in_Canada| job_level|Frecuency|
+--------------------+----------+---------+
|        CareerBeacon|Mid senior|     1798|
|        ClickJobs.io|Mid senior|     1707|
|Canadian Armed Fo...|Mid senior|      819|
|             Stantec|Mid senior|      807|
|Vancouver Coastal...|Mid senior|      681|
|      Walmart Canada|Mid senior|      589|
|     Randstad Canada|Mid senior|      448|
|GFL Environmental...| Associate|      421|
|CISSS de la Monté...|Mid senior|      400|
|      Energy Jobline|Mid senior|      394|
|Fraser Health Aut...|Mid senior|      392|
|   Jobs for Humanity|Mid senior|      392|
|Interior Health A...|Mid senior|      376|
|Savers | Value Vi...|Mid senior|      367|
|Loblaw Companies ...|Mid senior|      338|
|Alberta Health Se...|Mid senior|      337|
|BayCare Health Sy...| Associate|      285|
|                 MNP|Mid senior|      269|
|Shared Health-Soi...|Mid senior|      239|
|Little Burgundy S...|Mid senior

In [83]:
#The most commun job skills for Mid Senior
spark.sql("""
          select job_skills,
          count(job_skills) as Frecuency
          from df
          where job_level = 'Mid senior'
          group by job_skills
          order by job_skills asc
          """).show(50)

+--------------------+---------+
|          job_skills|Frecuency|
+--------------------+---------+
|                NULL|        0|
|".NET, C#, Web AP...|        1|
|"40 Act accountin...|        1|
|"ACD call managem...|        1|
|"ACLS, BLS, Nursi...|        1|
|"AF 65series inst...|        1|
|"AML/KYC verifica...|        1|
|"API650, API651, ...|        1|
|"API650, API651, ...|        1|
|"APRN Licensure, ...|        1|
|"ASME ""S"" and "...|        1|
|"Ability to commu...|        1|
|"Abrasive blast, ...|        1|
|"Access control s...|        1|
|"Accessibility, S...|        1|
|"Account Manageme...|        1|
|"Account Manageme...|        1|
|"Accounting conce...|        1|
|"Accounting, Audi...|        1|
|"Accounting, Audi...|        1|
|"Accounting, Audi...|        1|
|"Accounting, Audi...|        1|
|"Accounting, Audi...|        1|
|"Accounting, Busi...|        1|
|"Accounting, Econ...|        1|
|"Accounting, Fina...|        1|
|"Accounting, Fina...|        1|
|"Accounti